# Great Weapon Fighting
These are some math examples of the values you get for the Great Weapon Fighting style in 5e D&D
which let you reroll 1 or 2 on any weapon damage dice.

## Doing Some Math

In [77]:
import numpy as np
import pandas as pd

# Pulling this model from AceCalhoon's answer
# https://rpg.stackexchange.com/questions/47172/how-much-damage-does-great-weapon-fighting-add-on-average

series = {}
for i in range(4, 13, 2):
    series[i] = 1 - (2/i)
df = pd.DataFrame(series.items(), columns=['Die Type', 'Difference'])
df

,Die Type,Difference
0,4,0.500000
1,6,0.666667
2,8,0.750000
3,10,0.800000
4,12,0.833333


## Code to generate the values

In [78]:
def damage_roll(sides, num_die=1, gwf=False, crit_on=20):
    """
    This rolls a number of die and returns the result, rerolling 1s or 2s if gwf is true.
    :param sides: int sided die to use
    :param num_die: int number of die
    :param gwf: bool to indicate using gwf rerolls
    :param crit_on: int of value or above on d20 to become a crit.
    :return: int of roll total
    """
    # Crits double the number of rolls
    is_crit = np.random.randint(1, 21) >= crit_on
    num_actual = num_die * 2 if is_crit else num_die

    result = 0
    for _ in range(num_actual):
        roll = np.random.randint(1, sides + 1) # high is exlusive
        if gwf and roll < 3:
            roll = np.random.randint(1, sides + 1)
        result = roll + result

    if not is_crit and result > sides * num_die:
        print('error')

    return result

## Running the simulation for various die types.

In [79]:
# Setup a means to generate the full data.
def generate_weapon_data(gwf=False):
    crit_range = {'Base': 100, '20': 20, '19': 19}
    die_range = {'1d8': (8, 1), '1d10': (10, 1), '1d12': (12, 1), '2d6': (6, 2)}
    weapon_data = {}
    for cl, cv in crit_range.items():
        weapon_data[cl] = {}
        for dk, dv in die_range.items():
            weapon_data[cl][dk] = pd.Series(damage_roll(dv[0], dv[1], gwf=gwf, crit_on=cv) for _ in range(10000)).mean()
    return weapon_data

### Normal (No GWF)

In [80]:
normal_df = pd.DataFrame(generate_weapon_data())
normal_df

,Base,20,19
1d8,4.5202,4.7484,4.9670
1d10,5.5271,5.8200,6.0459
1d12,6.4163,6.7893,7.1729
2d6,6.9722,7.3274,7.7487


### With GWF

In [81]:
gwf_df = pd.DataFrame(generate_weapon_data(gwf=True))
gwf_df

,Base,20,19
1d8,5.2438,5.4589,5.7791
1d10,6.2585,6.5987,6.9968
1d12,7.2915,7.8369,8.0611
2d6,8.3164,8.7306,9.0978


## Various House Rules